In [15]:
import pandas as pd
import os
import json
import traceback
from dotenv import load_dotenv
load_dotenv()

from src.mcqgenerator.utils import read_file,get_table_data
from src.mcqgenerator.looger import logging

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    google_api_key = os.getenv("GOOGLE-API-KEY"),
    temperature = 0.7,
)

quiz_generation_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content = "You are an expert MCQ quiz maker. You have to create the quiz based on user input and given instructions."
        ),
        HumanMessage(
            content = """
                Please create {number} muliple choice questions in a {tone} tone based on the following text.
                **Text:**
                {text}
                
                **Instructions:**
                1. Ensure that all questions are from given text only.
                2. Ensure that the questions are not repeated.
                3. Format your entire response as a JSON object that strictly follows below given scheme.
                {response_json}
            """
        )
    ]    
)

quiz_chain = quiz_generation_prompt | llm | StrOutputParser()

review_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content ="You are an expert MCQ Analyzer for student benefit. You have to evaluate the given quiz as per instructions given below"
        ),
        HumanMessage(
            content="""Please review the multiple choice quiz intendede for {subject} sy=tudents.
                **Quiz for review:**
                '''json
                {quiz}
                '''

                **Your Task:**
                1. Evaluate the compleity of the questions.
                2. Assess if the quiz is appropriate for students cognitive and quantitative ability.
                3. New questions and answers created must be from given text only. The text is:{text}
                4. If the quiz is not suitable, then rewrite the questions and adjust the tone to perfectly fit students abilities.
                5. Provide a brief analysis of your changes or your approval of the original quiz.
                
            """
        )
    ]
)

review_chain = review_prompt | llm | StrOutputParser()

genereate_evaluate_chain =(
    RunnablePassthrough.assign(quiz = quiz_chain) | 
    RunnablePassthrough.assign(review = review_chain)
)



In [25]:
import os
import json
import traceback
import streamlit as st
import pandas as pd
from src.mcqgenerator.utils import read_file
from src.mcqgenerator.MCQGenerator import genereate_evaluate_chain
from src.mcqgenerator.looger import logging
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv
load_dotenv()

with open('Response.json','r') as file:
    RESPONSE_JSON = json.load(file)

In [26]:
file_path = "P:\Code\Gen_AI_course_ineuron\MCQ_GEN_AI\data.txt"
with open(file_path,"r") as file:
    TEXT = file.read()
uploaded_file = TEXT
mcq_count = 5
subject = "Machine Learning"
tone= "Simple"


<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\lenovo\AppData\Local\Temp\ipykernel_28044\3794405900.py:1: SyntaxWarning: invalid escape sequence '\C'
  file_path = "P:\Code\Gen_AI_course_ineuron\MCQ_GEN_AI\data.txt"


In [18]:
with get_openai_callback() as cb:
                response = genereate_evaluate_chain.invoke(
                    {
                        "text": TEXT,
                        "number":mcq_count,
                        "subject": subject,
                        "tone":tone,
                        "response_json": json.dumps(RESPONSE_JSON)
                    }
                )

In [19]:
print(response)

{'text': 'The Dawn of a New Intelligence: An In-Depth Exploration of Machine Learning\nIntroduction: The Silent Revolution\nIn the 21st century, a quiet but profound revolution is reshaping our world. It operates behind the screens of our devices, influences the products we buy, powers the entertainment we consume, and even helps doctors diagnose diseases. This revolution is driven by Machine Learning (ML), a powerful branch of artificial intelligence (AI) that, at its core, is about teaching computers to learn from data without being explicitly programmed for every task.\n\nFor centuries, human intelligence was the sole driver of innovation and problem-solving. We built machines to automate physical labor, following precise, human-written instructions. A calculator, for instance, performs complex arithmetic, but it only does what its programming dictates. It cannot learn, adapt, or discover a new mathematical theorem on its own. Machine learning shatters this paradigm. Instead of prov

In [32]:
def get_table_data(quiz_str):
    try:
        # Clean the string from markdown formatting
        cleaned_str = quiz_str.strip().lstrip("```json").rstrip("```").strip()
        
        # This variable will be a list of dictionaries
        quiz_list = json.loads(cleaned_str) 
        quiz_table_data = []

        # Iterate directly over the list
        for item in quiz_list:
            mcq = item.get("question")
            options = " || ".join(item.get("options", []))
            answer = item.get("answer")
            
            quiz_table_data.append({"MCQ": mcq, "Choices": options, "Answer": answer})

        return quiz_table_data

    except Exception as e:
        traceback.print_exc()
        return False

In [34]:
print(f"Total Tokens Used: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost:.4f}")
if isinstance(response,dict):
                quiz = response.get("quiz",None)    
                if quiz is not None:
                    table_data = get_table_data(quiz)
                    print(table_data)
                    if table_data is not None:
                        df = pd.DataFrame(table_data)
                        df.index += 1
                        

Total Tokens Used: 3169
Prompt Tokens: 265
Completion Tokens: 516
Total Cost (USD): $0.0000
[{'MCQ': 'According to the text, what is Artificial Intelligence (AI)?', 'Choices': 'Intelligence displayed by animals || Natural intelligence displayed by humans || Intelligence demonstrated by machines || The study of intelligent agents', 'Answer': 'Intelligence demonstrated by machines'}, {'MCQ': 'How do leading AI textbooks define the field of AI?', 'Choices': "As the study of human intelligence || As the study of natural intelligence || As the study of 'intelligent agents' || As the study of advanced web search engines", 'Answer': "As the study of 'intelligent agents'"}, {'MCQ': 'Which of the following is NOT mentioned as an application of AI in the provided text?', 'Choices': 'Advanced web search engines || Recommendation systems || Developing new programming languages || Self-driving cars', 'Answer': 'Developing new programming languages'}]


In [35]:
print(df)

                                                 MCQ  \
1  According to the text, what is Artificial Inte...   
2  How do leading AI textbooks define the field o...   
3  Which of the following is NOT mentioned as an ...   

                                             Choices  \
1  Intelligence displayed by animals || Natural i...   
2  As the study of human intelligence || As the s...   
3  Advanced web search engines || Recommendation ...   

                                  Answer  
1  Intelligence demonstrated by machines  
2   As the study of 'intelligent agents'  
3   Developing new programming languages  
